## Caching 缓存

LangChain为LLMs提供了可选的缓存层。这很有用，原因有两个：

- 如果您经常多次请求相同的完成，它可以通过减少您对 LLM 提供程序进行的 API 调用次数来节省资金。
- 它可以通过减少您对 LLM 提供程序进行的 API 调用次数来加速您的应用程序。

In [3]:
# 启动本地大模型并调用
from langchain_openai import ChatOpenAI

api_key = "xxx"
base_url = "http://localhost:1234/v1"

prompt = "请问两只兔子多少条腿?"

chat = ChatOpenAI(api_key=api_key, base_url=base_url, temperature=0.7)

chat.invoke(prompt)

AIMessage(content='两只兔子共有八条腿。每只兔子有四条腿，所以两只就是四条乘以二等于八条腿。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 36, 'total_tokens': 64, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'qwen2.5-32b-instruct', 'system_fingerprint': 'qwen2.5-32b-instruct', 'finish_reason': 'stop', 'logprobs': None}, id='run-5e45bc38-7baf-4580-8746-bcaf95d4108c-0', usage_metadata={'input_tokens': 36, 'output_tokens': 28, 'total_tokens': 64, 'input_token_details': {}, 'output_token_details': {}})

In [11]:
%%time
# 使用内存作为缓存
from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache

set_llm_cache(InMemoryCache())

CPU times: user 76 µs, sys: 1 µs, total: 77 µs
Wall time: 79.2 µs


In [9]:
%%time
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

output_parser = StrOutputParser()
prompt = ChatPromptTemplate.from_template("请根据主题写一篇今日头条的营销文案，要求携带emoji：{topic}")

chain = prompt | chat | output_parser

chain.invoke({"topic": "波司登羽绒服"})

CPU times: user 5.86 ms, sys: 1.36 ms, total: 7.22 ms
Wall time: 9.58 s


'❄️冬日温暖守护，尽在波司登羽绒服！🌟\n\n当寒风来袭，一件优质的羽绒服就是你的贴心小棉袄。波司登，作为国内知名的羽绒服品牌，不仅传承了精湛的制作工艺，更融入了现代科技和时尚元素，旨在为每一位顾客带来无与伦比的温暖体验。👫\n\n精选高品质鹅绒鸭绒填充，保证每一寸面料都能锁住暖意，让你在寒冬中也能感受到春天般的温暖。不仅如此，波司登羽绒服还注重设计感，无论是经典款还是潮流款，总有一款能满足你的个性化需求。🎨\n\n在这个冬天，给自己一个温暖的承诺吧！穿上波司登羽绒服，让寒冷远离你身边，与温暖同行！🌈\n\n立即行动，享受冬日里的小确幸，把温暖带回家！🛍️\n\n#波司登羽绒服 #冬季保暖专家'

In [10]:
%%time
chain.invoke({"topic": "波司登羽绒服"})


CPU times: user 1.1 ms, sys: 179 µs, total: 1.27 ms
Wall time: 1.28 ms


'❄️冬日温暖守护，尽在波司登羽绒服！🌟\n\n当寒风来袭，一件优质的羽绒服就是你的贴心小棉袄。波司登，作为国内知名的羽绒服品牌，不仅传承了精湛的制作工艺，更融入了现代科技和时尚元素，旨在为每一位顾客带来无与伦比的温暖体验。👫\n\n精选高品质鹅绒鸭绒填充，保证每一寸面料都能锁住暖意，让你在寒冬中也能感受到春天般的温暖。不仅如此，波司登羽绒服还注重设计感，无论是经典款还是潮流款，总有一款能满足你的个性化需求。🎨\n\n在这个冬天，给自己一个温暖的承诺吧！穿上波司登羽绒服，让寒冷远离你身边，与温暖同行！🌈\n\n立即行动，享受冬日里的小确幸，把温暖带回家！🛍️\n\n#波司登羽绒服 #冬季保暖专家'

## SQLite 缓存

In [14]:
from langchain.cache import SQLiteCache
from langchain.globals import set_llm_cache
import os

root_dir = os.path.dirname(os.getcwd())
db_path = os.path.join(root_dir, "db", "langchain.db")
os.makedirs(os.path.dirname(db_path), exist_ok=True)

set_llm_cache(SQLiteCache(database_path=db_path))



In [17]:
%%time
for chunk in chain.stream({"topic": "旺仔小馒头"}):
    print(chunk, end="", flush=True)

🌟【甜蜜“咬”一口，唤醒童年的味蕾】🌟

你是否还记得小时候那份纯真的快乐与期盼？每一颗小小的旺仔小馒头，都承载着满满的回忆和幸福的味道。🎈

今天，让我们一起回到那个充满欢笑的年代，打开一包旺仔小馒头，让那熟悉而诱人的香气带你穿越时光隧道，找回那些无忧无虑的日子。🌈

一口一个，酥软香甜，每一口都是对童年最美好的怀念。不论是忙碌工作后的小憩，还是与家人朋友相聚时的美好分享，旺仔小馒头总能带来满满的幸福感和温暖。💖

别让回忆只留在过去，现在就来一份，让这份甜蜜继续陪伴你成长的每一天！🌟

#旺仔小馒头 #童年回忆 #幸福的味道CPU times: user 250 ms, sys: 63 ms, total: 314 ms
Wall time: 8.1 s


In [19]:
%%time
for chunk in chain.stream({"topic": "旺仔小馒头"}):
    print(chunk, end="", flush=True)


标题：✨【甜蜜来袭】一口一个的幸福味道，旺仔小馒头让你爱不释手！🍬

正文：
亲爱的小伙伴们👋，是不是还在为找不到心仪的零食而烦恼呢？别急，今天要给大家带来一款绝对能俘获你心的小惊喜——旺仔小馒头！🌟

想象一下，当你拿起一包旺仔小馒头，那精致小巧的外观就像是一个个小小的糖果城堡🏰。轻轻一口咬下去，外皮酥脆，内里绵软，甜而不腻，每一口都是满满的幸福感。😋

不仅如此，每一块小馒头都蕴含着精心挑选的原料和独特配方，保证了它不仅好吃，而且健康营养，是大人小孩都喜欢的小零食！🍎👶

现在就来尝一尝这份来自旺仔的经典美味吧！无论是作为日常的小零嘴还是送给朋友的最佳手信，都是不二之选哦！💌

快来抢购你的专属甜蜜吧！🎉

标签：#旺仔小馒头 #经典美味 #幸福味道CPU times: user 279 ms, sys: 59.2 ms, total: 338 ms
Wall time: 10.1 s


In [21]:
%%time
for chunk in chain.stream({"topic": "clip-reid与agent结合在抓捕逃犯场景的应用"}):
    print(chunk, end="", flush=True)


标题：警务科技新突破！clip-reid与Agent联手，精准定位逃犯🎯🔍

正文：
在一个风起云涌的夜晚，城市中的每一盏路灯都见证了正义的力量。当逃犯们以为自己能够逃脱法律的手掌时，一个新的英雄组合已经悄然出现，那就是结合了最新AI技术的Clip-ReID与智能追踪系统Agent！⚡️

想象一下，你是一名侦查员，在茫茫人海中寻找那个只有一张模糊照片的目标人物。这是个挑战，但不再是不可能的任务。Clip-ReID利用其先进的图像识别能力，能够快速分析监控视频中的每个人脸和身形特征，而Agent则像是一个聪明的侦探助手，它能根据这些信息构建逃犯的行为模式，并预测他们的下一步行动路线。

当这两者强强联合时，抓捕工作变得前所未有的精确与高效。Clip-ReID负责在人群中“锁定”目标，而Agent则通过分析数据，计算出最优的追踪和拦截策略。这样的组合不仅大大缩短了案件侦破的时间，还提高了抓捕的成功率。

更重要的是，这种科技的应用为警力资源的分配提供了更加科学合理的依据，确保每一处需要加强监控的地方都能得到及时的关注和支持。在保障公共安全的同时，也减少了警察同志们的工作负担，让他们能够更专注于案件的核心问题上。

让我们一起期待这个警务科技的新时代吧！Clip-ReID与Agent的完美结合，正在为维护社会安宁贡献着不可替代的力量。未来，它们将继续成长和进化，帮助我们创造一个更加和谐、安全的社会环境。✨

结束语：
正义不会缺席！感谢Clip-ReID与Agent为我们带来的安全保障，让我们共同期待科技在警务工作中发挥更大作用的美好未来吧！🌟CPU times: user 651 ms, sys: 161 ms, total: 812 ms
Wall time: 17.3 s
